In [107]:
# initializations

import numpy as np

In [108]:
# data loading and preprocessing

with open('breast-cancer-wisconsin.data', 'r') as f:
    # removing the rows with missing values represented by '?'
    data_raw = [l.strip('\n').split(',') for l in f if '?' not in l]
# storing the data values as integers
data = np.array(data_raw).astype(int)

# data contains 683 out of 699 instances of the original data and 11 features

In [109]:
# hyperparameters

target_depth = 6 # max depth allowed for the decision tree with pruning
threshold_list = range(1, 11) # list of thresholds to be considered for splitting continuous features
part_one_feature = [4] # list of features to be considered for part 1
feature_list = [8, 10, 6, 9, 7, 2] # list of features to be considered for part 2


In [110]:
# helper functions for part 1
# Attribution: Hongtao Hao's sample solution for P2

def entropy(data):
    ''' Calculates the entropy of a given dataset'''
    entropy = 0
    count = len(data) # total number of instances
    n2 = np.sum(data[:, -1] == 2) # number of k1
    n4 = np.sum(data[:, -1] == 4) # number of k2
    if n2 == 0 or n4 == 0: 
        return 0
    else: 
        for n in [n2, n4]:
            p = n/count
            entropy += - (p * np.log2(p))
        return entropy

def infogain(data, feature, threshold):
    ''' Calculates the information gain for a given feature and threshold'''
    count = len(data)
    d1 = data[data[:, feature - 1] <= threshold]
    d2 = data[data[:, feature - 1] > threshold]
    proportion_d1 = len(d1) / count
    proportion_d2 = len(d2) / count
    return entropy(data) - proportion_d1 * entropy(d1) - proportion_d2 * entropy(d2)


def get_best_split(data, feature_list, threshold_list):
    ''' Calculates Max Info Gain, computes the threshold and returns the feature, threshold, and predictions for left and right nodes'''
    c = len(data)
    
    # c0 is the number of instances with class label 2
    c0 = sum(b[-1] == 2 for b in data)

    # if all instances have class label 2, return 2
    # else if all instances have class label 4, return 4
    if c0 == c: return 2, None, None, None
    if c0 == 0: return 4, None, None, None

    # compute possible information gain for all features and thresholds
    # pairwise combinations
    ig = [[infogain(
        data, feature, threshold) for threshold in threshold_list] for feature in feature_list]
    
    # convert ig to numpy array
    ig = np.array(ig)
    
    # find the maximum information gain
    max_ig = max(max(i) for i in ig)

    # if max_ig is 0, return 2 if c0 >= c - c0, else return 4
    # remember c0 is the number of instances with class label 2
    # and c - c0 is the number of instances with class label 4
    if max_ig == 0:
        if c0 >= c - c0:
            return 2, None, None, None
        else:
            return 4, None, None, None

    # can also return max_ig in case you need it for debugging
    
    # find the index of the maximum information gain
    idx = np.unravel_index(np.argmax(ig, axis=None), ig.shape)

    # return the feature, threshold, and predictions for left and right nodes
    feature, threshold = feature_list[idx[0]], threshold_list[idx[1]]

    # binary split: split the data into two parts based on the threshold
    dl = data[data[:, feature - 1] <= threshold]
    dr = data[data[:, feature - 1] > threshold]

    # get the number of instances with class label 2 and 4 in the left node
    dl_n2 = np.sum(dl[:,-1] == 2)
    dl_n4 = np.sum(dl[:,-1] == 4)

    # if the number of instances with class label 2 is greater than or equal to 4, predict 2
    if dl_n2 >= dl_n4:
        dl_prediction = 2
    else:
        # else predict 4
        dl_prediction = 4
    
    # get the number of instances with class label 2 and 4 in the left node
    dr_n2 = np.sum(dr[:,-1] == 2)
    dr_n4 = np.sum(dr[:,-1] == 4)

    # if the number of instances with class label 2 is greater than or equal to 4, predict 2
    if dr_n2 >= dl_n4:
        dr_prediction = 2
    else:
        # else predict 4
        dr_prediction = 4
    return feature, threshold, dl_prediction, dr_prediction

In [111]:
# question 1

n2 = np.sum(data[:,-1] == 2)
n4 = np.sum(data[:,-1] == 4)
print('Number of instances with class label 2: ', n2)
print('Number of instances with class label 4: ', n4)

Number of instances with class label 2:  444
Number of instances with class label 4:  239


In [112]:
# question 2

print('Entropy of the dataset: ', entropy(data))

Entropy of the dataset:  0.9340026588217948


In [113]:
feature = part_one_feature[0] - 1
optimal_threshold = None
max_ig = 0
for threshold in threshold_list:
    ig = infogain(data, feature, threshold)
    if ig > max_ig:
        max_ig = ig
        optimal_threshold = threshold

print('Number of 2s with feature value <= optimal threshold: ', np.sum(data[data[:, feature] <= optimal_threshold][:,-1] == 2))
print('Number of 2s with feature value > optimal threshold: ', np.sum(data[data[:, feature] > optimal_threshold][:,-1] == 2))
print('Number of 4s with feature value <= optimal threshold: ', np.sum(data[data[:, feature] <= optimal_threshold][:,-1] == 4))
print('Number of 4s with feature value > optimal threshold: ', np.sum(data[data[:, feature] > optimal_threshold][:,-1] == 4))

Number of 2s with feature value <= optimal threshold:  395
Number of 2s with feature value > optimal threshold:  49
Number of 4s with feature value <= optimal threshold:  9
Number of 4s with feature value > optimal threshold:  230


In [114]:
# question 4

print('Information gain after split', infogain(data, 4, optimal_threshold))

Information gain after split 0.5690253924531676


In [115]:
# helper functions for part 2
# Attribution: Hongtao Hao's sample solution for P2

class Node:
    def __init__(self, feature = None, threshold = None, l_prediction = None, r_prediction = None):
        self.feature = feature
        self.threshold = threshold
        self.l_prediction = l_prediction # prediction for left subtree
        self.r_prediction = r_prediction # prediction for right subtree
        self.l = None # left child or left subtree
        self.r = None # right child or right subtree`
        self.correct = 0 # number of correct predictions

def split(data, node):
    # split the data into two parts based on the threshold
    feature, threshold = node.feature, node.threshold
    d1 = data[data[:,feature-1] <= threshold]
    d2 = data[data[:,feature-1] > threshold]
    return (d1,d2)

def create_tree(data, node, feature_list):
    ''' Recursively creates the tree'''
    d1,d2 = split(data, node)
    f1, t1, l1_prediction, r1_prediction = get_best_split(d1, feature_list, threshold_list)
    f2, t2, l2_prediction, r2_prediction = get_best_split(d2, feature_list, threshold_list)
    if t1 == None: 
        node.l_pre = f1
    else:
        node.l = Node(f1, t1, l1_prediction, r1_prediction)
        create_tree(d1, node.l, feature_list)
    if t2 == None: 
        node.r_pre = f2
    else:
        node.r = Node(f2, t2, l2_prediction, r2_prediction)
        create_tree(d2, node.r, feature_list)  
    
def maxDepth(node):
    ''' Calculates the maximum depth of the tree'''
    if node is None:
        return 0 ;
    else :
        left_depth = maxDepth(node.l)
        right_depth = maxDepth(node.r)
 
        return max(left_depth, right_depth) + 1

def expand_root(data, feature_list, threshold_list):
    ''' Expands the root node'''
    feature, threshold, dl, dr = get_best_split(
        data, feature_list, threshold_list)
    root = Node(feature, threshold)
    # first split
    data1, data2 = split(data, root)
    create_tree(data, root, feature_list)
    return root

def tree_prediction(node, x):
    ''' Predicts the class label for a single instance (test data)'''
    feature = node.feature
    threshold = node.threshold
    l_prediction = node.l_prediction
    r_prediction = node.r_prediction
    l = node.l
    r = node.r

    if x[feature-1] <= threshold:
        if l_prediction == x[-1]:
            node.correct += 1
        if l == None:
            return l_prediction
        else:
            return tree_prediction(l, x)
    else:
        if r_prediction == x[-1]:
            node.correct += 1
        if r == None:
            return r_prediction
        else:
            return tree_prediction(r, x)

def print_tree(node, f, prefix=''):
    fea = node.feature
    t = node.threshold
    l_pre = node.l_prediction
    r_pre = node.r_prediction
    l = node.l
    r = node.r
    if l == None:
        f.write(prefix+'if (x'+str(fea)+') <= '+str(t)+') return '+str(l_pre)+'\n')
    else:
        f.write(prefix+'if (x'+str(fea)+') <= '+str(t)+')\n')
        print_tree(l, f, prefix+' ')
    if r == None:
        f.write(prefix+'else return '+str(r_pre)+'\n')
    else:
        f.write(prefix+'else\n')
        print_tree(r, f, prefix+' ')
                

def prune(node, depth):
    ''' Prunes the tree to the specified depth'''
    if depth == 1:
        node.l = None
        node.r = None
    else:
        if node.l != None:
            prune(node.l, depth-1)
        if node.r != None:
            prune(node.r, depth-1)

In [116]:
# question 5

# create a decision tree with the root node
root = expand_root(data, feature_list, threshold_list)
# print the tree to a file
with open('tree.txt', 'w') as f:
    print_tree(root, f)


In [117]:
# question 6

print('Maximum depth of the tree: ', maxDepth(root))

Maximum depth of the tree:  11


In [118]:
# question 7

# load the test data
with open('test.txt', 'r') as f:
    test_raw = [l.strip('\n').split(',') for l in f if '?' not in l]
test = np.array(test_raw).astype(int)

# get all the predictions
predictions = [tree_prediction(root, x) for x in test]

# store the predictions in a file
with open('predictions.txt', 'w') as f:
    f.write(','.join([str(x) for x in predictions]))

In [119]:
# question 8

# prune the tree to the specified depth and store the pruned tree in a file
prune(root, target_depth)
with open('pruned_tree.txt', 'w') as f:
    print_tree(root, f)

In [120]:
# question 9

# get all the predictions on the pruned tree
predictions = [tree_prediction(root, x) for x in test]

# store the predictions in a file
with open('pruned_predictions.txt', 'w') as f:
    f.write(','.join([str(x) for x in predictions]))